In [4]:
import pandas as pd
from tokenizers import ByteLevelBPETokenizer

In [25]:
model_checkpoint = 't5-small'
model_name='sparql-translator-t5-2021-06-20'
model_path='../../data/tokenizers/'+model_name
ds_path='../../data/dataset/lc-quad-wikidata-2021-06-20'
tokenizer_dir = '../../data/tokenizers/lc-quad-wikidata-2021-06-20'

In [14]:
path = '../../data/lc-quad-2021-06-20.txt'

In [2]:
from datasets import load_dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [5]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

In [9]:
raw_datasets['test']['translation'][0]

{'en': 'Who is the  {country} for {head of state} of {Mahmoud Abbas}',
 'sparql': 'select distinct ?sbj where { ?sbj wdt:P35 wd:Q127998 . ?sbj wdt:P31 wd:Q6256 }'}

In [18]:
# Create PLAIN text from csv files
MyFile=open(path,'w')
for d in raw_datasets['test']['translation']:
    MyFile.write(d['sparql'])
    MyFile.write('\n')
for d in raw_datasets['train']['translation']:
    MyFile.write(d['sparql'])
    MyFile.write('\n')
MyFile.close()


In [83]:
# from tokenizers import AddedToken
# tokenizer.add_tokens(['slck_token' , AddedToken("select", normalized=False)]) #'<slct>'

In [21]:
tokenizer.train(files=path, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [26]:
!mkdir {tokenizer_dir}
tokenizer.save_model(tokenizer_dir)

['../../data/tokenizers/lc-quad-wikidata-2021-06-20/vocab.json',
 '../../data/tokenizers/lc-quad-wikidata-2021-06-20/merges.txt']

In [27]:
sample = 'SELECT ?uri WHERE {?uri dbo:publisher dbr:GMT_Games}'.lower()
t = tokenizer.encode(sample)
print(t.ids)
print(t.tokens)

[277, 261, 880, 77, 274, 271, 35, 880, 77, 337, 3437, 30, 84, 1774, 13194, 272, 337, 1967, 30, 75, 13208, 67, 75, 23065, 97]
['select', 'Ġ?', 'ur', 'i', 'Ġwhere', 'Ġ{', '?', 'ur', 'i', 'Ġd', 'bo', ':', 'p', 'ub', 'lis', 'her', 'Ġd', 'br', ':', 'g', 'mt', '_', 'g', 'ames', '}']


In [28]:
tokenizer.decode([274, 281, 262, 267, 279, 270, 35, 267, 268, 30, 819, 291, 30, 1358, 67, 753, 97])

' where p w .anssbj? . wdt:583al:1331_344}'

In [89]:
# If you want to tokenize '\n' as a special token you can simply do:
# newline_token = tokenizers.AddedToken("\n", normalized=False)
# my_tokenizer.add_tokens(newline_token)

In [90]:
# Code to load your saved tokenizer from the json is:
# from transformers import PreTrainedTokenizerFast
# tokenizerx = PreTrainedTokenizerFast(tokenizer_file= '../data/tokenizer-2021-05-30/vocab.json')